### Cài đặt thư viện

In [1]:
import os
import cv2
import numpy as np
import joblib
import json
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### 🧠 Bước 1: Trích xuất đặc trưng (7D vector từ model onnx)

In [2]:
def extract_softmax_feature(img_path, recognizer):
    img = cv2.imread(img_path)
    if img is None:
        print("❌ Lỗi đọc ảnh:", img_path)
        return None

    img = cv2.resize(img, (112, 112))
    img = img.astype(np.float32)

    result = recognizer.infer(img)

    return result  # output softmax (7 lớp)

### 🗂️ Bước 2: Gộp dữ liệu các class và trích feature

In [3]:
def build_dataset(images_dir, recognizer):
    X = []
    y = []
    label_names = []

    class_names = sorted([
        name for name in os.listdir(images_dir)
        if os.path.isdir(os.path.join(images_dir, name))
    ])
    label_names = class_names  # Gán 1 lần duy nhất

    for idx, class_name in enumerate(class_names):
        class_dir = os.path.join(images_dir, class_name)

        for fname in os.listdir(class_dir):
            if not fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
                continue
            path = os.path.join(class_dir, fname)
            feat = extract_softmax_feature(path, recognizer)
            if feat is not None:
                X.append(feat)
                y.append(idx)

    return np.array(X), np.array(y), label_names

### 🎯 Bước 3: Train classifier 8 lớp

In [4]:
def train_classifier(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)

    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"✅ Accuracy: {acc:.4f}")
    return model

### 💾 Bước 4: Lưu model classifier & label names


In [5]:
def save_model(model, label_names, out_dir='../../model/facial_expression_recognition'):
    os.makedirs(out_dir, exist_ok=True)
    joblib.dump(model, os.path.join(out_dir, 'svc_facial_expression_classifier.pkl'))
    with open(os.path.join(out_dir, 'facial_expression_label_names.json'), 'w') as f:
        json.dump(label_names, f)
    print("✅ Model và nhãn đã được lưu.")

### 🔁 Bước 5: Inference

In [6]:
def predict_emotion(image_path, recognizer, classifier, label_names):
    feat = extract_softmax_feature(image_path, recognizer)
    if feat is None:
        return "Unknown"
    pred = classifier.predict([feat])[0]
    return label_names[pred]

In [10]:
# ==== Load ONNX model ====
from facial_fer_model import FacialExpressionRecog  # from OpenCV Zoo

onnx_path = "../../model/facial_expression_recognition_mobilefacenet_2022july.onnx"
recognizer = FacialExpressionRecog(onnx_path)
images_dir = "../../images/facial_expression_recognition"  # Thư mục chứa các thư mục con: Happy/, Sleepy/, ...
list_images = [
    "happy.jpg",  # Thay bằng ảnh thử của bạn
    "sleepy.jpg",  # Thay bằng ảnh thử của bạn
    # "angry.jpg",  # Thay bằng ảnh thử của bạn
    "sad.jpg",  # Thay bằng ảnh thử của bạn
    # "surprise.jpg",  # Thay bằng ảnh thử của bạn
    # "disgust.jpg",  # Thay bằng ảnh thử của bạn
    # "fear.jpg",  # Thay bằng ảnh thử của bạn
]

if __name__ == "__main__":
    
    print("📁 Loading dataset...")
    X, y, label_names = build_dataset(images_dir, recognizer)

    print("🎓 Training classifier...")
    model = train_classifier(X, y)

    print("💾 Saving model...")
    save_model(model, label_names)

    # Inference
    print("🔍 Inference...")
    for test_img in list_images:
        if os.path.exists(test_img):
            pred = predict_emotion(test_img, recognizer, model, label_names)
            print(f"🧠 Dự đoán cảm xúc cho ảnh '{test_img}': {pred}")
        else:
            print("📸 File test không tồn tại — bỏ qua inference.")

📁 Loading dataset...
🎓 Training classifier...
✅ Accuracy: 0.9084
💾 Saving model...
✅ Model và nhãn đã được lưu.
🔍 Inference...
🧠 Dự đoán cảm xúc cho ảnh 'happy.jpg': Sleepy
🧠 Dự đoán cảm xúc cho ảnh 'sleepy.jpg': Sleepy
🧠 Dự đoán cảm xúc cho ảnh 'sad.jpg': Happy
